# 5. Function Calling

In [3]:
import sys
from pathlib import Path
import warnings
import os

warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    try:
        if "google.colab" in str(get_ipython()):
            return True
    except:
        pass
    return False

def clone_repository() -> None:
    # Check if repository already exists
    repo_dir = Path("pm25-forecast-openmeteo-aqicn")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd pm25-forecast-openmeteo-aqicn
    else:
        print("Cloning repository...")
        !git clone https://github.com/KristinaPalmquist/pm25-forecast-openmeteo-aqicn.git
        %cd pm25-forecast-openmeteo-aqicn

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

def fix_numpy_compatibility():
    print("Fixing numpy compatibility for hopsworks/pandas...")
    try:
        # Use precompiled wheels with compatible versions
        !pip install --force-reinstall numpy==1.24.4 pandas==2.0.3
        print("Numpy and pandas fixed. Please restart runtime and run again.")
    except Exception as e:
        print(f"Fix attempt failed: {e}")
        print("Please manually restart runtime and try again.")

if is_google_colab():
    try:
        import numpy
        numpy.array([1, 2, 3])
        import pandas as pd
        print("Basic packages working correctly")

        clone_repository()
        install_dependencies()

        import hopsworks
        print("All packages working correctly")

        root_dir = str(Path().absolute())
        print("Google Colab environment")
        
    except (ValueError, ImportError) as e:
        if "numpy.dtype size changed" in str(e) or "numpy.strings" in str(e) or "numpy" in str(e).lower():
            fix_numpy_compatibility()
            raise SystemExit("Please restart runtime (Runtime > Restart runtime) and run the notebook again.")
        else:
            raise

else:
    root_dir = Path().absolute()
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

from utils import config

if is_google_colab():
    from google.colab import userdata
    import hopsworks
    project = hopsworks.login(
        api_key_value=userdata.get('HOPSWORKS_API_KEY'),
        engine="python"
    )
    AQICN_API_KEY = userdata.get('AQICN_API_KEY')
    
else:
    # Local development - use .env file
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn


ModuleNotFoundError: No module named 'pydantic'

In [ ]:
import sys
print("Python executable:", sys.executable)
print("Python path:", sys.path[:3])  # First few paths

In [ ]:
from xgboost import XGBRegressor
import hopsworks
from openai import OpenAI
from utils.llm_chain import (
    load_model, 
    get_llm_chain, 
    generate_response, 
    generate_response_openai,
)
import pandas as pd
import os
import warnings
from utils import config
from utils import airquality
import json
from datetime import datetime, timedelta
import time

warnings.filterwarnings("ignore")

In [ ]:
if is_google_colab():
    fs = project.get_feature_store()
    secrets = hopsworks.get_secrets_api()
else:
    HOPSWORKS_API_KEY = getattr(settings, 'HOPSWORKS_API_KEY', None)

    if HOPSWORKS_API_KEY is not None and hasattr(HOPSWORKS_API_KEY, 'get_secret_value'):
        HOPSWORKS_API_KEY = HOPSWORKS_API_KEY.get_secret_value()

    project = hopsworks.login(engine="python", api_key_value=HOPSWORKS_API_KEY)

    fs = project.get_feature_store()

    secrets = hopsworks.get_secrets_api()
    AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value


today = datetime.today().date()

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

Set SENSOR_CSV_FILE in .env with the relative path to a sensor to process it, or leave it unset to process all sensors in the `data` folder

In [ ]:
if is_google_colab():
    sensor_csv_file = None
else:
    sensor_csv_file = getattr(settings, 'SENSOR_CSV_FILE', None)

if sensor_csv_file:
    # Read one secret for single sensor mode
    _, _, _, _, _, sensor_id = airquality.read_sensor_data(sensor_csv_file)
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id}"
    location_str = secrets.get_secret(secret_name).value
    locations = {sensor_id: json.loads(location_str)}
else:
    # Read all individual secrets in batch mode
    all_secrets = secrets.get_secrets()
    locations = {}
    for secret in all_secrets:
        if secret.name.startswith("SENSOR_LOCATION_JSON_"):
            sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
            location_str = secrets.get_secret(secret.name).value
            if location_str:
                locations[sensor_id] = json.loads(location_str)



In [ ]:
mr = project.get_model_registry()

MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

retrieved_models = {}

for sensor_id in locations.keys():
    if len(retrieved_models) > 0:
        continue
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    retrieved_model = None

    available_models = mr.get_models(name=model_name)
    if available_models:
        retrieved_model = max(available_models, key=lambda model: model.version)

    saved_model_dir = retrieved_model.download()
    model_air_quality = XGBRegressor()
    model_air_quality.load_model(saved_model_dir + "/model.json")
    fv = retrieved_model.get_feature_view()
    
    retrieved_models[sensor_id] = fv, model_air_quality


## LLM Loading

In [ ]:
import time
start_time = time.time()

# Load the LLM and its corresponding tokenizer
model_llm, tokenizer = load_model(model_id="imiraoui/OpenHermes-2.5-Mistral-7B-sharded")

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

## LangChain Setup

In [ ]:
start_time = time.time()

llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

## Domain Specific Questions

In [ ]:
feature_view, model_air_quality = retrieved_models[list(retrieved_models.keys())[0]]

In [ ]:
QUESTION7 = "Hi!"
response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION1 = "What was the average air quality from 2024-01-10 till 2024-01-14?"

response1 = generate_response(
    QUESTION1, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response1)

In [ ]:
QUESTION11 = "When and what was the air quality like last week?"

response11 = generate_response(
    QUESTION11, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response11)

In [ ]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14?"

response12 = generate_response(
    QUESTION12, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response12)

In [ ]:
QUESTION2a = "What was the air quality like last week?"

response2 = generate_response(
    QUESTION2a,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION2 = "What was the air quality like yesterday?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION3 = "What will the air quality be like next Tuesday?"

response3 = generate_response(
    QUESTION3, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response3)

In [ ]:
QUESTION4 = "What will the air quality be like the day after tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response4)

In [ ]:
QUESTION5 = "What will the air quality be like this Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response5)

In [ ]:
QUESTION7 = "What will the air quality be like for the rest of the week?"

response7 = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Will the air quality be safe or not for the next week?"

response = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Is tomorrow's air quality level dangerous?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
from xgboost import XGBRegressor
from functions.llm_chain import load_model, get_llm_chain, generate_response


In [ ]:
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    if y.ndim > 1 and y.shape[1] > 1:
        y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def generate_query_response(user_query, method, openai_api_key=None):
    if method == 'Hermes LLM':        
        response = generate_response(
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            model_llm,
            tokenizer,
            llm_chain,
            verbose=False,
        )
        return response
    
    elif method == 'OpenAI API' and openai_api_key:
        client = OpenAI(
            api_key=openai_api_key
        )
        
        response = generate_response_openai(   
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            client=client,
            verbose=True,
        )
        return response
        
    else:
        return "Invalid method or missing API key."

def handle_input(text_input=None, audio_input=None, method='Hermes LLM', openai_api_key=""):
    if audio_input is not None:
        user_query = transcribe(audio_input)
    else:
        user_query = text_input
    
    # Check if OpenAI API key is required but not provided
    if method == 'OpenAI API' and not openai_api_key.strip():
        return "OpenAI API key is required for this method."

    if user_query:
        return generate_query_response(user_query, method, openai_api_key)
    else:
        return "Please provide input either via text or voice."
    

# Setting up the Gradio Interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type here or use voice input..."), 
        gr.Audio(), 
        gr.Radio(["Hermes LLM", "OpenAI API"], label="Choose the response generation method"),
        gr.Textbox(label="Enter your OpenAI API key (only if you selected OpenAI API):", type="password")  # Removed `optional=True`
    ],
    outputs="text",
    title="🌤️ AirQuality AI Assistant 💬",
    description="Ask your questions about air quality or use your voice to interact. Select the response generation method and provide an OpenAI API key if necessary."
)

iface.launch(share=True)
